In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, BatchNormalization, Input, Add, GlobalAveragePooling2D, Attention
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import (classification_report, confusion_matrix, f1_score,
                           precision_recall_curve, roc_curve, roc_auc_score,
                           average_precision_score, cohen_kappa_score,
                           matthews_corrcoef, balanced_accuracy_score,
                           jaccard_score, hamming_loss, recall_score)
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import pandas as pd

# **Run it in the last**

In [ ]:
csv_path = r"C:\path\to\your\data.csv"
HEIGHT = 128
WIDTH = 128
CHANNELS = 1

BATCHSIZE = 256
EPOCHS = 200
subset = float(input("Subset (e.g., 0.05 for 5%) [default=1]: ") or 1)
train_ratio = float(input("Train split (e.g., 0.8) [default=0.8]: ") or 0.8)

model, history, X_test, y_test = train_model_from_csv(csv_path, total_subset=subset, train_size=train_ratio)
plot_training_history(history)
metrics = evaluate_model(model, X_test, y_test)

In [ ]:
def load_data_csv(csv_path):
    df = pd.read_csv(csv_path)
    labels = df["label"].values
    features = df.drop(columns=["label"]).values
    # Normalize and reshape into 2D images
    features = features.astype("float32") / 255.0
    features = features.reshape(-1, HEIGHT, WIDTH, CHANNELS)
    return features, labels

In [ ]:
def evaluate_model(model, X_test, y_test):
    y_pred = (model.predict(X_test) > 0.5).astype(int)
    y_pred_proba = model.predict(X_test)
    
    print("\n=== Detailed classification report ===")
    print(classification_report(y_test, y_pred))
    
    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig('confusion_matrix.png')
    plt.close()
    
    # ROC Curve
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc="lower right")
    plt.savefig('roc_curve.png')
    plt.close()
    
    # Precision-Recall Curve
    precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
    average_precision = average_precision_score(y_test, y_pred_proba)
    
    plt.figure(figsize=(8, 6))
    plt.plot(recall, precision, label=f'PR curve (AP = {average_precision:.2f})')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend(loc="lower left")
    plt.savefig('pr_curve.png')
    plt.close()
    
    metrics = {
        'accuracy': balanced_accuracy_score(y_test, y_pred),
        'precision': average_precision_score(y_test, y_pred),
        'recall': recall_score(y_test, y_pred),
        'f1': f1_score(y_test, y_pred),
        'roc_auc': roc_auc,
        'average_precision': average_precision,
        'kappa': cohen_kappa_score(y_test, y_pred),
        'mcc': matthews_corrcoef(y_test, y_pred),
        'jaccard': jaccard_score(y_test, y_pred),
        'hamming_loss': hamming_loss(y_test, y_pred)
    }
    
    print("\n=== All metrics ===")
    print(f"Balanced Accuracy: {metrics['accuracy']:.4f}")
    print(f"Precision: {metrics['precision']:.4f}")
    print(f"Recall: {metrics['recall']:.4f}")
    print(f"F1 Score: {metrics['f1']:.4f}")
    print(f"ROC AUC Score: {metrics['roc_auc']:.4f}")
    print(f"Average Precision: {metrics['average_precision']:.4f}")
    print(f"Cohen's Kappa: {metrics['kappa']:.4f}")
    print(f"Matthews Correlation Coefficient: {metrics['mcc']:.4f}")
    print(f"Jaccard Score: {metrics['jaccard']:.4f}")
    print(f"Hamming Loss: {metrics['hamming_loss']:.4f}")
    
    return metrics

In [ ]:
def attention_block(x, filters):
    # Self-attention mechanism
    attention = Attention()([x, x])
    x = Add()([x, attention])
    return x

In [ ]:
def create_model(input_shape=(224, 224,1)):
    inputs = Input(shape=input_shape)
    
  # Block 1
    x = Conv2D(32, (3, 3), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = MaxPooling2D((2, 2))(x)
  
  # Block 2
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = MaxPooling2D((2, 2))(x)
  
  # Block 3
    x = Conv2D(128, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = MaxPooling2D((2, 2))(x)
  
  # Optional: Attention here
  # x = attention_block(x, 128)
  
  # Block 4
    x = Conv2D(256, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = MaxPooling2D((2, 2))(x)
  
  # Head
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu', kernel_regularizer=l2(0.001))(x)
    x = Dropout(0.3)(x)
    output = Dense(1, activation='sigmoid')(x)

    return Model(inputs, output)

In [ ]:
def train_model_from_csv(csv_path, total_subset=1, train_size=0.8, test_size=0.2):
    if train_size + test_size != 1.0:
        raise ValueError("train_size and test_size must sum to 1 when not using validation.")
    
    images, labels = load_data_csv(csv_path)
    images, labels = shuffle(images, labels, random_state=42)

    # Subset
    subset_size = int(len(images) * total_subset)
    images, labels = images[:subset_size], labels[:subset_size]

    # Split
    X_train, X_test, y_train, y_test = train_test_split(
        images, labels, test_size=test_size, random_state=42
    )

    model = create_model(input_shape=(HEIGHT, WIDTH, CHANNELS))

    model.compile(
        optimizer=Adam(learning_rate=0.00005, clipnorm=1.0),
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.1),
        metrics=['accuracy', 
                 tf.keras.metrics.Precision(), 
                 tf.keras.metrics.Recall(), 
                 tf.keras.metrics.F1Score()]
    )

    def lr_schedule(epoch):
        if epoch < 50:
            return 0.00005
        elif epoch < 100:
            return 0.00001
        else:
            return 0.000005

    callbacks = [
        ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True),
        EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True),
        LearningRateScheduler(lr_schedule)
    ]

    history = model.fit(
        X_train, y_train,
        epochs=EPOCHS,
        batch_size=BATCHSIZE,
        validation_data=(X_test, y_test),
        callbacks=callbacks
    )

    return model, history, X_test, y_test

In [ ]:
def plot_training_history(history):
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.close()